# CDO Ocean/Land Mask


In this notebook we demonstrate how to mask ocean or land:

- Look at the data information
- Create land and ocean masks to limit the field to land/ocean values 
- Visualise data using xarray and hvplot


This example uses the Coupled Model Intercomparison Project (CMIP6) collections. 

---
inspired by the notebook in  https://github.com/NCI-data-analysis-platform/climate-cmip.git
- Original Authors: NCI Virtual Research Environment Team
- Keywords: CMIP, CDO, land mask, ocean mask
- Create Date: 2019-Dec; Update Date: 2021-Feb
---
Adapted to work on the DKRZ jupyterhub environment: S. Kindermann August 2022

This notebook is licenced under the [Creative Commons Attribution 4.0 International license](https://creativecommons.org/licenses/by/4.0/)

### Load CDO module

To load the CDO module on the VDI, run:

``` $ module load cdo```

### CMIP6 data

The following example file includes the near surface temperature from the 20th century all-forcing historical simulation based on the CESM2.0 model of NCAR. 

In [ ]:
!ls /pool/data/CMIP6/data/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc

### Have a look at the data file using cdo info

Basic usage: **cdo info filename | head -xx**

**head** displays only the first lines of the output (in this case xx lines)

In [ ]:
 !cdo info /pool/data/CMIP6/data/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc| head -15

### Let's see which years this file includes

We use the function **showyear** to display all the years in this file.

**Basic usage**: **cdo showyear file**

In [ ]:
!cdo showyear /pool/data/CMIP6/data/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc

### Create topography data 

```
$ cdo -f <format of the output file (e.g. nc/grb/srv/ext/ieg)> topo topo.nc
```
The operator topo creates a half degree regular grid with heights and depths. This can be very useful for generating masks to a certain height/depth.

In [ ]:
### create output directory if it doesn't already exist
import os
outdir = './output'
if not os.path.exists(outdir):
    os.mkdir(outdir)

!cdo -f nc topo ./output/topo.nc 

Let's visualise our topo.nc file in xarray

In [ ]:
import xarray as xr 
topo= xr.open_dataset("./output/topo.nc")
topo.topo.plot()              

From our topo.nc file we can observe that the topography values over land areas are greater than 0 whereas over ocean areas the topography are less than 0.

### Create ocean mask file

**Basic usage**: 

```cdo gtc, 0 -remapcon, input.nc topo.nc output.nc```


There are several steps involved in here:

First, remapping the topo.nc file to change its resolution so that it is consistent with the input data that we use: 

```
$ cdo -remapcon,/pool/data/CMIP6/data/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc ./output/topo.nc ./output/topo_adjusted.nc
```

Second, create a mask containing 1 if the topography element is greater than 0: 
```
$ cdo gtc,0 topo_adjusted.nc seamask.nc
```

### Now we will try to chain all the commands together

**Basic Usage:** 

```cdo -f <file_type> -gtc,0 -remapcon,<input.nc> -topo <output.nc>```


In [ ]:
!cdo -f nc -gtc,0 -remapcon,/pool/data/CMIP6/data/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc -topo ./output/seamask.nc

**Similarly, we can get land mask using the following command:**

```cdo -f <file_type> -lec,0 -remapcon,<input.nc> -topo <output.nc>```

In [ ]:
!cdo -f nc -lec,0 -remapcon,/pool/data/CMIP6/data/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc -topo ./output/landmask.nc

### Let's have a look at what the generated seamask.nc looks like

In [ ]:
sea = xr.open_dataset("./output/seamask.nc")
sea.topo.plot()

From our topography file, we can see that all ocean values are 0 and all land values are 1.

### Now we can apply the seamask to the original data to mask out ocean data

Multiply input.nc and seamask, and the ocean values in output.nc all become zero.

```cdo mul <input.nc> <seamask.nc> <output.nc>```

Set zero as the missing value in the data:

```cdo setmissval,0 <input.nc> <output.nc>```


To combine these two commands together:

```cdo setmissval,0 -mul <input.nc> <seamask.nc> <output.nc>**```
``` 

Warning: Be cautious when using zero as the missing value in case zero has already been used as a reasonable element in the original data. In this example, the data use Kelvin(K) as the unit of temperature and therefore using zero as the missing value is not an issue as 0deg K cannot be interpreted as a reasonable Earth near surface temperature.

In [ ]:
!cdo setmissval,0 -mul /pool/data/CMIP6/data/CMIP/NCAR/CESM2/historical/r1i1p1f1/Amon/tas/gn/v20190308/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc ./output/seamask.nc ./output/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412_land.nc

### Let's have a look at the new land data file

In [ ]:
# to do: ncview shows nice plot .. :-)
import xarray as xr
import hvplot.xarray
land = xr.open_dataset("./output/tas_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412_land.nc")

#land.tas.isel(time=0).plot()
land.tas.hvplot.quadmesh(width=600)

### Summary

In this example, we show how to use CDO to create land/ocean masks when we only need to visualize the ocean/land data.

## Reference

https://code.zmaw.de/projects/cdo/embedded/cdo.pdf
